# [6.3 Regression Planning](http://artint.info/2e/html/ArtInt2e.Ch6.S3.html)
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=89) (page 89)

## About
In a regression planner, a node is a subgoal that need to be achieved. This is different than in a forward planner, where a node is instead state.

## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter*. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [ ]:
from aipython.searchProblem import Arc, Search_problem
from aipython.searchProblem import Search_problem
from aipython.stripsRegressionPlanner import Subgoal


class Regression_STRIPS(Search_problem):
    """A search problem where:
    * a node is a goal to be achieved, represented by a set of propositions.
    * the dynamics are specified by the STRIPS representation of actions
    """

    def __init__(self, planning_problem, heur=lambda s, g: 0):
        """creates a regression seach space from a planning problem.
        heur(state,goal) is a heuristic function; 
           an underestimate of the cost from state to goal, where
           both state and goals are feature:value dictionaries
        """
        self.prob_domain = planning_problem.prob_domain
        self.top_goal = Subgoal(planning_problem.goal)
        self.initial_state = planning_problem.initial_state
        self.heur = heur

    def is_goal(self, subgoal):
        """if subgoal is true in the initial state, a path has been found"""
        goal_asst = subgoal.assignment
        return all((g in self.initial_state) and (self.initial_state[g] == goal_asst[g])
                   for g in goal_asst)

    def start_node(self):
        """the start node is the top-level goal"""
        return self.top_goal

    def neighbors(self, subgoal):
        """returns a list of the arcs for the neighbors of subgoal in this problem"""
        cost = 1
        goal_asst = subgoal.assignment
        return [Arc(subgoal, self.weakest_precond(act, goal_asst), cost, act)
                for act in self.prob_domain.actions
                if self.possible(act, goal_asst)]

    def possible(self, act, goal_asst):
        """True if act is possible to achieve goal_asst.

        the action achieves an element of the effects and
        the action doesn't delete something that needs to be achieved and
        the precoditions are consistent with other subgoals that need to be achieved
        """
        effects = self.prob_domain.strips_map[act].effects
        preconds = self.prob_domain.strips_map[act].preconditions
        return (any(goal_asst[prop] == effects[prop]
                    for prop in effects if prop in goal_asst)
                and all(goal_asst[prop] == effects[prop]
                        for prop in effects if prop in goal_asst)
                and all(goal_asst[prop] == preconds[prop]
                        for prop in preconds if prop not in effects and prop in goal_asst)
                )

    def weakest_precond(self, act, goal_asst):
        """returns the subgoal that must be true so goal_asst holds after act"""
        new_asst = self.prob_domain.strips_map[act].preconditions.copy()
        for g in goal_asst:
            if g not in self.prob_domain.strips_map[act].effects:
                new_asst[g] = goal_asst[g]
        return Subgoal(new_asst)

    def heuristic(self, subgoal):
        """in the regression planner a node is a subgoal.
        the heuristic is an (under)estimate of the cost of going from the initial state to subgoal.
        """
        return self.heur(self.initial_state, subgoal.assignment)

In [ ]:
from aipython.searchBranchAndBound import DF_branch_and_bound
from aipython.stripsProblem import problem0, simple_problem1, simple_problem2, blocks1, blocks2, blocks3
from IPython.display import display


d = DF_branch_and_bound(Regression_STRIPS(simple_problem1), 10)
d.search()
display(d)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
d.search()

The regression planner can also use a heuristic function, just as with the forward planner. However, just because a heuristic is useful for a forward planner does not mean it is useful for a regression planner, and vice versa.

In [ ]:
from aipython.searchMPP import SearcherMPP
from aipython.stripsProblem import problem0, simple_problem1, simple_problem2, blocks1, blocks2, blocks3
from aipython.stripsHeuristic import heuristic_fun
from IPython.display import display

m = SearcherMPP(Regression_STRIPS(simple_problem1, heuristic_fun))
m.show_node_heuristics = True
m.search()
display(m)

**Note**: Run the method below to find _additional_ solutions. You should _not_ run this until you have already finished finding a solution because it will cause the state of the frontier to be indeterminate. You will know when you have found a solution when the output says "n paths have been expanded".

In [ ]:
m.search()